In [164]:
import requests
import pandas as pd
from pandas import json_normalize
pd.set_option('display.max_columns', None)

In [165]:
# API endpoint
url = "https://api-v3.mbta.com/routes"

# Query parameters
params = {
    "filter[type]": "0,1"
}

# Make the GET request
response_1 = requests.get(url, params=params)

In [166]:
response_1 = response_1.json()

In [167]:
df = pd.DataFrame(response_1['data'])

In [168]:
df_normalized = json_normalize(df['attributes'])

In [169]:
df_routes = pd.concat([df, df_normalized], axis=1)
df_routes

,attributes,id,links,relationships,type,color,description,direction_destinations,direction_names,fare_class,long_name,short_name,sort_order,text_color,type
0,"{'color': 'DA291C', 'description': 'Rapid Tran...",Red,{'self': '/routes/Red'},"{'line': {'data': {'id': 'line-Red', 'type': '...",route,DA291C,Rapid Transit,"[Ashmont/Braintree, Alewife]","[South, North]",Rapid Transit,Red Line,,10010,FFFFFF,1
1,"{'color': 'DA291C', 'description': 'Rapid Tran...",Mattapan,{'self': '/routes/Mattapan'},"{'line': {'data': {'id': 'line-Mattapan', 'typ...",route,DA291C,Rapid Transit,"[Mattapan, Ashmont]","[Outbound, Inbound]",Rapid Transit,Mattapan Trolley,,10011,FFFFFF,0
2,"{'color': 'ED8B00', 'description': 'Rapid Tran...",Orange,{'self': '/routes/Orange'},"{'line': {'data': {'id': 'line-Orange', 'type'...",route,ED8B00,Rapid Transit,"[Forest Hills, Oak Grove]","[South, North]",Rapid Transit,Orange Line,,10020,FFFFFF,1
3,"{'color': '00843D', 'description': 'Rapid Tran...",Green-B,{'self': '/routes/Green-B'},"{'line': {'data': {'id': 'line-Green', 'type':...",route,00843D,Rapid Transit,"[Boston College, Government Center]","[West, East]",Rapid Transit,Green Line B,B,10032,FFFFFF,0
4,"{'color': '00843D', 'description': 'Rapid Tran...",Green-C,{'self': '/routes/Green-C'},"{'line': {'data': {'id': 'line-Green', 'type':...",route,00843D,Rapid Transit,"[Cleveland Circle, Government Center]","[West, East]",Rapid Transit,Green Line C,C,10033,FFFFFF,0
5,"{'color': '00843D', 'description': 'Rapid Tran...",Green-D,{'self': '/routes/Green-D'},"{'line': {'data': {'id': 'line-Green', 'type':...",route,00843D,Rapid Transit,"[Riverside, Union Square]","[West, East]",Rapid Transit,Green Line D,D,10034,FFFFFF,0
6,"{'color': '00843D', 'description': 'Rapid Tran...",Green-E,{'self': '/routes/Green-E'},"{'line': {'data': {'id': 'line-Green', 'type':...",route,00843D,Rapid Transit,"[Heath Street, Medford/Tufts]","[West, East]",Rapid Transit,Green Line E,E,10035,FFFFFF,0
7,"{'color': '003DA5', 'description': 'Rapid Tran...",Blue,{'self': '/routes/Blue'},"{'line': {'data': {'id': 'line-Blue', 'type': ...",route,003DA5,Rapid Transit,"[Bowdoin, Wonderland]","[West, East]",Free,Blue Line,,10040,FFFFFF,1


In [170]:
#Question 1
print(df_routes['long_name'])

0            Red Line
1    Mattapan Trolley
2         Orange Line
3        Green Line B
4        Green Line C
5        Green Line D
6        Green Line E
7           Blue Line
Name: long_name, dtype: object


In [171]:
def get_stops(route):
    url = "https://api-v3.mbta.com/stops"
    # Query parameters
    params = {
        "filter[route]": route,
    }
    # Make the GET request
    response = requests.get(url, params=params).json()['data']
    stops = [stop['id'] for stop in response]
    return [route,len(stops)]


In [172]:
#Question 2
#1
#get the route ids
ids = list(df_routes["id"])
print(ids)
#get all stops based on route id?
for route in ids:
    print(get_stops(route))
#print(get_stops('Mattapan'))

['Red', 'Mattapan', 'Orange', 'Green-B', 'Green-C', 'Green-D', 'Green-E', 'Blue']
['Red', 22]
['Mattapan', 8]
['Orange', 20]
['Green-B', 23]
['Green-C', 20]
['Green-D', 25]
['Green-E', 25]
['Blue', 12]


In [173]:
def get_all_stops_with_routes(route):    
    url = "https://api-v3.mbta.com/stops"
    # Query parameters
    params = {
        "include": "route",        
        "filter[route]": route,
        "sort": "id"
    }
    # Make the GET request
    response = requests.get(url, params=params).json()['data']
    df = pd.DataFrame(response)
    df_normalized = json_normalize(df['relationships'])
    df_ = pd.concat([df, df_normalized], axis=1)
    all_stops = df_[['id','route.data.id']]
    all_stops = all_stops.to_dict(orient='records')
    return all_stops
    

In [179]:
#q2 #3
#need to save all stops for all line with route relationship included
#then join the dataframes by stop id to determine what stops serve > 1 routes 
print(ids)
#get all stops based on route id?
all_stops_with_route = []
for route in ids:
    all_stops_with_route.extend(get_all_stops_with_routes(route))
print(len(all_stops_with_route))
df = pd.DataFrame(all_stops_with_route)
df_grouped = df.groupby('id')['route.data.id'].apply(list).reset_index()
filtered = df_grouped[df_grouped['route.data.id'].str.len() > 1]
filtered

['Red', 'Mattapan', 'Orange', 'Green-B', 'Green-C', 'Green-D', 'Green-E', 'Blue']
155


,id,route.data.id
6,place-armnl,"[Green-B, Green-C, Green-D, Green-E]"
7,place-asmnl,"[Red, Mattapan]"
19,place-boyls,"[Green-B, Green-C, Green-D, Green-E]"
40,place-coecl,"[Green-B, Green-C, Green-D, Green-E]"
44,place-dwnxg,"[Red, Orange]"
54,place-gover,"[Green-B, Green-C, Green-D, Green-E, Blue]"
57,place-haecl,"[Orange, Green-D, Green-E]"
62,place-hymnl,"[Green-B, Green-C, Green-D]"
65,place-kencl,"[Green-B, Green-C, Green-D]"
69,place-lech,"[Green-D, Green-E]"


In [180]:
#question 3
